In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DICT_MESES = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}


In [3]:
path_limpos = "Tratados"
path_brutos = "Brutos"

In [38]:
path_ena_bruto = path_brutos+"\\Simples_Energia_Natural_Afluente_Subsistema_Barra__data.csv"
path_carga_bruto = path_brutos+"\\Simples_Carga_de_Energia_Barra_Mês_data.csv"
path_vazao_bruto = path_brutos+"\\vazao_mensal_juazeiro_sf.csv"

In [65]:
ena_bruto = pd.read_csv(path_ena_bruto)
carga_bruto = pd.read_csv(path_carga_bruto)
vazao_bruto = pd.read_csv(path_vazao_bruto,on_bad_lines='skip',sep=";")

In [6]:
datas_brutas_ena = ena_bruto.iloc[:,0]
datas_brutas_carga= carga_bruto.iloc[:,0]

In [7]:
def limpa_datas_ena_carga(datas_brutas, DICT_MESES):

    df_limpo = pd.DataFrame()
    meses_brutos = datas_brutas.apply(lambda x: x.split(" ")[0])
    anos_brutos = datas_brutas.apply(lambda x: x.split(" ")[1])

    df_limpo["Month"] = meses_brutos.apply(lambda x: DICT_MESES[x])
    df_limpo["Year"] = anos_brutos

    df_limpo["Data"] = pd.to_datetime(df_limpo[["Month", "Year"]].assign(day=1))
    df_limpo.drop(["Month","Year"],axis=1,inplace=True)

    return df_limpo

In [8]:
df_ena_limpo = limpa_datas_ena_carga(datas_brutas_ena, DICT_MESES)
df_carga_limpo = limpa_datas_ena_carga(datas_brutas_carga, DICT_MESES)

In [9]:
df_ena_limpo["ENA"] = ena_bruto.iloc[:,-1]
df_carga_limpo["Carga"] = carga_bruto.iloc[:,-1]


In [10]:
df_carga_limpo = df_carga_limpo[df_carga_limpo.Data.dt.year >= 2002]
df_carga_limpo

,Data,Carga
0,2023-06-01,39917.917295
1,2023-05-01,41246.681730
2,2023-04-01,40279.185284
3,2023-03-01,43688.792105
4,2023-02-01,43029.632306
...,...,...
253,2002-05-01,24774.471236
254,2002-04-01,25772.850000
255,2002-03-01,25133.201223
256,2002-02-01,23452.702202


In [11]:
df_ena_limpo.sort_values(by="Data",inplace=True)
df_carga_limpo.sort_values(by="Data",inplace=True)

In [12]:
df_ena_limpo.to_csv(path_limpos+"\\ena_limpo.csv",index=False)
df_carga_limpo.to_csv(path_limpos+"\\carga_limpo.csv",index=False)

In [74]:
vazao_limpo = vazao_bruto[["Data","Media"]]
vazao_limpo.loc[:,"Data"] = pd.to_datetime(vazao_limpo.loc[:,"Data"],dayfirst=True)
vazao_limpo.sort_values(by="Data",ascending=True,inplace=True)
mask_datas = vazao_limpo["Data"]>=pd.to_datetime("01/01/2000")
vazao_limpo = vazao_limpo[mask_datas]
vazao_limpo.columns=["Data","Vazao"]
vazao_limpo.reset_index(inplace=True,drop=True)

vazao_limpo.Vazao = pd.to_numeric(vazao_limpo.Vazao.str.replace(",","."))
vazao_limpo

,Data,Vazao
0,2000-01-01,2156.77
1,2000-02-01,1897.96
2,2000-03-01,1879.81
3,2000-04-01,2303.92
4,2000-05-01,2068.45
...,...,...
267,2022-06-01,952.94
268,2022-07-01,1626.89
269,2022-08-01,1786.46
270,2022-09-01,1306.63


In [77]:
vazao_limpo.sort_values(by="Data",inplace=True)
vazao_limpo.to_csv(path_limpos+"\\vazao_limpo.csv",index=False)